![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
## Step 2: Models Implementation

<hr style="border: 2px solid blue;">

### Step 2.1: XGBoost MultiOutputRegressor Regressor

In [ ]:
# drop columns that are not selected and the target columns
y = final_merged_df[['co2_avg', 'pm10_avg', 'pm25_avg', 'no2_avg']]
X = final_merged_df.drop(columns=['co2_avg', 'pm10_avg', 'pm25_avg', 'no2_avg'])

# drop aadt columns and keep vkm columns as they are highly correlated. vkm is calculated using aadt and speed columns
X.drop(columns=['aadt_motorcycle', 'aadt_taxi',
       'aadt_petrol_car', 'aadt_diesel_car', 'aadt_electric_car',
       'aadt_petrol_phv', 'aadt_diesel_phv', 'aadt_electric_phv',
       'aadt_petrol_lgv', 'aadt_diesel_lgv', 'aadt_electric_lgv',
       'aadt_hgvs_rigid_2_axles', 'aadt_hgvs_rigid_3_axles',
       'aadt_hgvs_rigid_4_or_more_axles', 'aadt_hgvs_articulated_3_to_4_axles',
       'aadt_hgvs_articulated_5_axles', 'aadt_hgvs_articulated_6_axles',
       'aadt_buses', 'aadt_coaches'], inplace=True)

In [ ]:
# save features list in a csv file
X.to_csv('df_final_features_list.csv', index=False)

In [ ]:
X.head()

In [ ]:
# final list of features used for training
selected_columns = X.columns
selected_columns

In [ ]:
# implement multi-output regression using xgboost regressor
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train the model
xgb = MultiOutputRegressor(XGBRegressor(objective='reg:squarederror', n_estimators=1000, 
                                        learning_rate=0.1, max_depth=6, random_state=42))
xgb.fit(X_train, y_train)

# predict the target values
y_pred = xgb.predict(X_test)

# calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')

In [ ]:
# save the model
import joblib

joblib.dump(xgb, 'xgb_model.pkl')

# load the model
xgb = joblib.load('xgb_model.pkl')

# predict the emissions for a new data
new_data = X_test.iloc[0]
new_data = new_data.values.reshape(1, -1)
y_pred = xgb.predict(new_data)
print(y_pred)

# compare the predicted values with the actual values
print(y_test.iloc[0])

In [ ]:
# display the predicted values and actual values in a dataframe
y_pred_df = pd.DataFrame(y_pred, columns=['co2_avg', 'pm10_avg', 'pm25_avg', 'no2_avg'])
y_test_df = y_test.reset_index(drop=True)
y_test_df = y_test_df.iloc[0].to_frame().T
y_test_df.columns = ['co2_avg', 'pm10_avg', 'pm25_avg', 'no2_avg']

# rename the columns to have 'actual' and 'predicted' as a prefix
y_pred_df = y_pred_df.add_prefix('predicted_')
y_test_df = y_test_df.add_prefix('actual_')

# concatenate the two dataframes
df_compare = pd.concat([y_test_df, y_pred_df], axis=1)
df_compare

In [ ]:
# Feature importance from RF model (sum importance across both targets)

# Extract feature importances for each target variable
feature_importances = np.mean([est.feature_importances_ for est in xgb.estimators_], axis=0)

# Get feature names
feature_names = X.columns

# Plot feature importance
plt.figure(figsize=(8, 5))
plt.barh(feature_names, feature_importances, color='skyblue')
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.title("Feature Importance in Key Pollutant Average Emission Prediction")
plt.show()

<hr style="border: 2px solid blue;">

### Step 2.2: Random Forest Multioutput Regressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
rf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
rf.fit(X_train, y_train)

In [ ]:
y_pred_rf = rf.predict(X_test)

In [ ]:
# 'co2_avg', 'pm10_avg', 'pm25_avg', 'no2_avg'
# Evaluate performance
mae_rf = mean_absolute_error(y_test, y_pred_rf, multioutput='raw_values')
r2_rf = r2_score(y_test, y_pred_rf, multioutput='raw_values')
mse_rf = mean_squared_error(y_test, y_pred_rf, multioutput='raw_values')
rmse_rf = np.sqrt(mse_rf)

mse = mean_squared_error(y_test, y_pred_rf)
rmse = np.sqrt(mse)

print(f"Random Forest - MAE: {mae_rf}")
print(f"Random Forest - R² Score: {r2_rf}")
print(f"Random Forest - MSE: {mse_rf}")
print(f"Random Forest - RMSE: {rmse_rf}")

print(f"Random Forest - MSE: {mse}")
print(f"Random Forest - RMSE: {rmse}")

**Interpretation**
1. Mean Absolute Error (MAE)

MAE:
[0.9121,0.0002,0.0001,0.0021]

    MAE measures the average absolute difference between actual and predicted values.
    Lower MAE values indicate better performance.
    The first target variable (co2_avg) has an MAE of 0.9121, meaning on average, the prediction error is around 0.91 units.
    The other target variables ('pm10_avg', 'pm25_avg', 'no2_avg') have very small MAE values, suggesting the model predicts them with high accuracy.

2. R² Score (Coefficient of Determination)

R² Scores:
[0.9860,0.9004,0.9446,0.9829]

    R² measures how well the model explains variance in the data.
    The closer R² is to 1, the better the model performance.
    co2_avg (first value: 0.9860): The model explains 98.4% of the variance in PM2.5 values.
    pm10_avg (second value: 0.9004): The model explains 90% of the variance in pm10_avg.
    The last two values ('pm25_avg', 'no2_avg') have slightly lower but still strong performance (0.9446 and 0.9829).
        0.90 indicates that 90% of the variance is explained by the model, which is good performance.

3. Mean Squared Error (MSE)

MSE:
[17.8751,0.000002,0.0000005,0.0001]

    MSE penalises larger errors more heavily than MAE since it squares the errors.
    Lower values mean better predictions.
    The first value (17.8751) is for co2_avg, suggesting some variation in predictions but still reasonably low.
    The other values are extremely small, indicating high precision for 'pm10_avg', 'pm25_avg', 'no2_avg'.


4. Root Mean Squared Error (RMSE)

RMSE:
[4.2279,0.0017,0.0007,0.0095]

    RMSE is the square root of MSE and gives an error measure in the same units as the data.
    The first value (4.2279) means that CO2 predictions have an average error of about 4.5 units.
    The other RMSE values are extremely small, indicating highly accurate predictions for SO₂ and other pollutants.


Overall Interpretation:

✅ Good performance

    R² values close to 1 suggest that the model explains almost all variance in the data.
    Low MAE and RMSE values indicate that the model makes highly accurate predictions.
    Small MSE for 'pm10_avg', 'pm25_avg', 'no2_avg' means the model is extremely precise for those variables.

⚠️ Potential areas for improvement:

    The first variable (co2_avg) has slightly higher RMSE (4.2279) than others. We also need to make sure that the other values are not subjects to overfitting. We could try to enhance this models by applying methods such as Hyperparameter tuning (adjust n_estimators, max_depth, etc.) or Feature engineering (adding more relevant input variables).

# Feature importance from RF model (sum importance across both targets)

# Extract feature importances for each target variable
feature_importances = np.mean([est.feature_importances_ for est in rf.estimators_], axis=0)

# Get feature names
feature_names = X.columns

# Plot feature importance
plt.figure(figsize=(8, 5))
plt.barh(feature_names, feature_importances, color='skyblue')
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.title("Feature Importance in Key Pollutant Average Emission Prediction")
plt.show()


<hr style="border: 2px solid blue;">

### Step 2.3: Ridge Multi-Output Regressor

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

In [ ]:
# load the merged dataframe from the csv file
final_merged_df = pd.read_csv('final_merged_df.csv')

In [ ]:
# load the final list of features from the csv file
final_features_df = pd.read_csv('df_final_features_list.csv')

# Display the first few rows
print("Merged Data:\n", final_merged_df.head())
print("\nFinal Features List:\n", final_features_df.head())

In [ ]:
print("Column Names in final_merged_df:")
print(final_merged_df.columns.tolist())

In [ ]:
# final list of features used for training
selected_columns = final_features_df.columns.tolist()
selected_columns

In [ ]:
# Ensure only selected features are used
X = final_merged_df[selected_columns]

# Define target variables
target_variables = ['co2_avg', 'pm10_avg', 'pm25_avg', 'no2_avg']


# Define target matrix (multi-output regression)
y = final_merged_df[target_variables]

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:

ridge = Ridge(alpha=1.0)
ridge.fit(X_train_scaled, y_train)

In [ ]:
import joblib
# Save the trained model
model_filename = "ridge_model_all_emissions.pkl"
joblib.dump((ridge, scaler, target_variables), model_filename)
print(f"Model saved: {model_filename}")

In [ ]:
# Load the saved model
loaded_ridge, loaded_scaler, loaded_targets = joblib.load(model_filename)
print(f"\n Loaded model for targets: {loaded_targets}")

In [ ]:
# Transform test data using saved scaler
X_test_scaled = loaded_scaler.transform(X_test)

In [ ]:
# Predict emissions for each target variable
y_pred = loaded_ridge.predict(X_test_scaled)

In [ ]:
# Convert predictions to DataFrame for evaluation
y_pred_df = pd.DataFrame(y_pred, columns=loaded_targets, index=y_test.index)

In [ ]:
results = {}

for target in loaded_targets:
    mse = mean_squared_error(y_test[target], y_pred_df[target])
    r2 = r2_score(y_test[target], y_pred_df[target])

    print(f"\nResults for {target}:")
    print(f"  - Mean Squared Error (MSE): {mse}")
    print(f"  - R² Score: {r2}")

    # Store results
    results[target] = {"MSE": mse, "R2": r2}

    # Visualization
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=y_test[target], y=y_pred_df[target], alpha=0.5)
    plt.xlabel(f"Actual {target}")
    plt.ylabel(f"Predicted {target}")
    plt.title(f"Ridge Regression Predictions vs Actual ({target})")
    plt.show()

In [ ]:
results_df = pd.DataFrame(results).T
print("\n Final Model Performance Summary:\n", results_df)

In [ ]:
# Extract feature importance from trained Ridge model
feature_importance = pd.DataFrame(loaded_ridge.coef_, columns=X.columns, index=loaded_targets)

# Plot feature importance for each emission type
for target in loaded_targets:
    plt.figure(figsize=(10, 6))
    sorted_coeffs = feature_importance.loc[target].sort_values(ascending=False)
    sorted_coeffs.plot(kind="bar", color="royalblue")
    plt.title(f"Feature Importance for {target}")
    plt.xlabel("Feature")
    plt.ylabel("Coefficient Value")
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# Define the parameter grid for alpha tuning
param_grid = {"alpha": [0.01, 0.1, 1, 10, 100]}

# Perform Grid Search with Cross-Validation
ridge_cv = GridSearchCV(Ridge(), param_grid, scoring="r2", cv=5)
ridge_cv.fit(X_train_scaled, y_train)

# Get the best alpha value
best_alpha = ridge_cv.best_params_["alpha"]
print(f"Best Alpha Value: {best_alpha}")

# Train Ridge Regression using the optimized alpha
optimized_ridge = Ridge(alpha=best_alpha)
optimized_ridge.fit(X_train_scaled, y_train)

# Save the optimized model
joblib.dump((optimized_ridge, scaler, target_variables), "optimized_ridge_model.pkl")
print("Optimized Ridge model saved.")